## Loading Data

In [1]:
import pandas as pd
import numpy as np 
from pymongo import MongoClient

df = pd.read_csv('Chicago Food Inspections.csv')

In [2]:
df.shape

(100049, 14)

In [3]:
df.isnull().sum()

Inspection ID          0
DBA Name               0
AKA Name             686
License                5
Facility Type        852
Risk                  21
Address                0
City                  72
State                 37
Zip                   12
Inspection Date        0
Inspection Type        1
Results                0
Violations         25006
dtype: int64

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100049 entries, 0 to 100048
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Inspection ID    100049 non-null  int64  
 1   DBA Name         100049 non-null  object 
 2   AKA Name         99363 non-null   object 
 3   License          100044 non-null  float64
 4   Facility Type    99197 non-null   object 
 5   Risk             100028 non-null  object 
 6   Address          100049 non-null  object 
 7   City             99977 non-null   object 
 8   State            100012 non-null  object 
 9   Zip              100037 non-null  float64
 10  Inspection Date  100049 non-null  object 
 11  Inspection Type  100048 non-null  object 
 12  Results          100049 non-null  object 
 13  Violations       75043 non-null   object 
dtypes: float64(2), int64(1), object(11)
memory usage: 10.7+ MB


## Data cleaning

In [5]:
df.columns = df.columns.str.strip()

df['AKA Name'].fillna('Unknown', inplace=True)
df['Facility Type'].fillna('Unknown', inplace=True)
df['Risk'].fillna('Unknown', inplace=True)
df['City'].fillna('Unknown', inplace=True)
df['State'].fillna('Unknown', inplace=True)
df['Inspection Type'].fillna('Unknown', inplace=True)
df['Violations'].fillna('None', inplace=True)

if 'License' in df.columns:
    df['License'].fillna(0, inplace=True) 
else:
    print("Column 'License' not found in the DataFrame.")

if 'Zip' in df.columns:
    df['Zip'].fillna(df['Zip'].median(), inplace=True)  
else:
    print("Column 'Zip' not found in the DataFrame.")

print("Null values after cleaning:")
print(df.isnull().sum())

Null values after cleaning:
Inspection ID      0
DBA Name           0
AKA Name           0
License            0
Facility Type      0
Risk               0
Address            0
City               0
State              0
Zip                0
Inspection Date    0
Inspection Type    0
Results            0
Violations         0
dtype: int64


## Connect to MongoDB

In [6]:
client = MongoClient("mongodb://localhost:27017/")
db = client['food_inspections_db']
collection = db['inspections']

# Insert data into MongoDB
collection.insert_many(df.to_dict(orient='records'))
print("Data successfully inserted into MongoDB.")

Data successfully inserted into MongoDB.


## Check the shape of the dataset

In [7]:
print(f"Rows: {df.shape[0]}, Columns: {df.shape[1]}")

Rows: 100049, Columns: 14


## Aggregated dataset

In [8]:
pipeline = [
    {"$group": {"_id": "$Inspection ID", "count": {"$sum": 1}}},  
    {"$sort": {"count": -1}},  
    {"$limit": 5} 
]

aggregated_data = collection.aggregate(pipeline)
aggregated_df = pd.DataFrame(list(aggregated_data))
print("\nTop 5 Records of Aggregated Data:")
print(aggregated_df)


Top 5 Records of Aggregated Data:
       _id  count
0  1588628      6
1  2081539      6
2  1464957      6
3  2286169      6
4  2009361      6
